# localBizdat 
### get data about local businesses (i.e. hotels, restaurants) surrounding the brewery

In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import requests
from config import gkey
import json
import numpy as np

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)

In [3]:
Base.prepare(engine, reflect=True)

In [4]:
session = Session(engine)

In [6]:
#create dataframe
brew_df = pd.read_sql_query("SELECT id, latitude, longitude FROM business", con=engine)
brew_df

,id,latitude,longitude
0,0,35.722939,-77.912148
1,1,35.578687,-82.594280
2,2,35.581549,-82.604400
3,3,35.200849,-83.824356
4,4,36.244335,-80.848763
...,...,...,...
271,271,36.104924,-80.245300
272,272,35.216754,-80.856676
273,273,35.910023,-79.068814
274,274,35.236705,-81.015616


In [7]:
#set base url for api call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [8]:
#create columns for new data from api call
brew_df['hotel_name']=''
brew_df['hotel_address']=''
brew_df['r1_name']=''
brew_df['r1_address']=''
brew_df['r2_name']=''
brew_df['r2_address']=''
brew_df['r3_name']=''
brew_df['r3_address']=''

In [9]:
#hit google places api for hotels

for index, row in brew_df.iterrows():
    loc = f"{str(row.latitude)},{str(row.longitude)}"
    params = {
        "location": loc,
        "type": "lodging",
        "rankby": "distance",
        "key": gkey,
    }
    
    response = requests.get(base_url, params=params)
    results = response.json()["results"]
    
    try:
        brew_df.at[index, 'hotel_name'] = results[0]["name"]
        brew_df.at[index, 'hotel_address'] = results[0]["vicinity"]

    except:
        pass

In [10]:
#hit google places api for restaurants

for index, row in brew_df.iterrows():
    loc = f"{str(row.latitude)},{str(row.longitude)}"
    params = {
        "location": loc,
        "type": "restaurant",
        "rankby": "distance",
        "key": gkey,
    }
    
    response = requests.get(base_url, params=params)
    results = response.json()["results"]
    
    try:
        brew_df.at[index, 'r1_name'] = results[0]["name"]
        brew_df.at[index, 'r1_address'] = results[0]["vicinity"]
        brew_df.at[index, 'r2_name'] = results[1]["name"]
        brew_df.at[index, 'r2_address'] = results[1]["vicinity"]
        brew_df.at[index, 'r3_name'] = results[2]["name"]
        brew_df.at[index, 'r3_address'] = results[2]["vicinity"]

    except:
        pass

In [11]:
brew_df

,id,latitude,longitude,hotel_name,hotel_address,r1_name,r1_address,r2_name,r2_address,r3_name,r3_address
0,0,35.722939,-77.912148,Heart of Wilson Motel,"501 Nash St W, Wilson",Ruckus & Redemption,"220 Barnes Street South Suite C, Wilson",Da Bayou,"124 Barnes Street Southwest #3902, Wilson",Snapper,"108 Barnes Street Southwest, Wilson"
1,1,35.578687,-82.594280,Bon Paul & Sharky's Hostel,"816 Haywood Road #3114, Asheville",Sunflower Diner,"771 Haywood Road, Asheville",Universal Joint,"784 Haywood Road, Asheville",Belly Up food truck,"760 Haywood Road, Asheville"
2,2,35.581549,-82.604400,Carriage House,"1413 Patton Avenue, Asheville",UpCountry Brewing Company,"1042 Haywood Road, Asheville",El Paraiso Mexican Grill,"1047 Haywood Road, Asheville",Tienda Los Nenes,"1341 Parkwood Road Suite 110, Asheville"
3,3,35.200849,-83.824356,Walker Inn,Andrews,Granieri's Italian Restaurant,"983 Main Street, Andrews",The Happy Hawg BBQ,"1130 Main Street, Andrews",Hoppy Trout Brewing Company,"911 Main Street, Andrews"
4,4,36.244335,-80.848763,Dereks,"245 Meshack Ln, Elkin",Angry Troll Brewing,"222 East Main Street, Elkin",The Smoothie Station,"150 East Market Street, Elkin",Southern On Main,"102 East Main Street, Elkin"
...,...,...,...,...,...,...,...,...,...,...,...
271,271,36.104924,-80.245300,APEX Mobile App,"550 North Liberty Street STE 125, Winston-Salem",Earl's,"121 West 9th Street, Winston-Salem",Famous Toastery,"770 Liberty View Court, Winston-Salem",Mary's Gourmet Diner,"723 N, Trade Street Northwest, Winston-Salem"
272,272,35.216754,-80.856676,Holiday Inn Express & Suites Charlotte - South...,"1617 South Tryon Street, Charlotte",Twisted Eats by Kre8,"1440 South Tryon Street #109, Charlotte",VANA,"1440 South Tryon Street Suite 100, Charlotte",Wooden Robot Brewery,"1440 South Tryon Street UNIT 110, Charlotte"
273,273,35.910023,-79.068814,Hampton Inn & Suites Chapel Hill-Carrboro/Down...,"370 East Main Street Unit 100, Carrboro",Amante Gourmet Pizza - Carrboro,"300 East Main Street, Carrboro",Oakleaf,"310 East Main Street, Carrboro",Rise Carrboro,"310 East Main Street, Carrboro"
274,274,35.236705,-81.015616,Wylie Overlook Manufactured Housing Community,"3 Greenway Circle, Belmont",White's Restaurant,"417 Catawba Street, Belmont",Hardee's,United States,Peace N' Hominy,"403 E Catawba Street #200, Belmont"


In [12]:
# add data to sqlite database
brew_df[['id','hotel_name','hotel_address','r1_name','r1_address','r2_name','r2_address','r3_name','r3_address']].to_sql('localbusinesses', con=engine, if_exists='replace')

In [13]:
#test sqlite data
engine.execute("SELECT * FROM localbusinesses").fetchall()[0]

(0, 0, 'Heart of Wilson Motel', '501 Nash St W, Wilson', 'Ruckus & Redemption', '220 Barnes Street South Suite C, Wilson', 'Da Bayou', '124 Barnes Street Southwest #3902, Wilson', 'Snapper', '108 Barnes Street Southwest, Wilson')